In [20]:
import numpy as np
import pickle
from gurobipy import *
import time
from collections import defaultdict
    

def solve_exact(r,p):
    """Solves the MCLP problem exactly using the Gurboi solver
    
    Credit: https://github.com/cyang-kth/maximum-coverage-location/blob/master/mclp.py

    Inputs: 
        r - radius for coverage 
        p - number of facilities to be sited

    """
    start = time.time()

    dtype = [('facility', int), ('population', float), ('cnt', int)]
    facility = np.genfromtxt('facility.csv', delimiter=',', skip_header=1)
    demand = np.genfromtxt('demand.csv', delimiter=',', skip_header=1)
    total_pop = np.genfromtxt('total_pop.csv', delimiter=',', skip_header=1, dtype=dtype)
    with open('covered.pickle', 'rb') as handle:
        coverage = pickle.load(handle)
    # convert to sets 
    dict_set = defaultdict(set)
    for k,v in coverage.items():
        for i in v: 
            dict_set[k].add(i)

    print(dict_set)
    
    distance_matrix = np.genfromtxt('distance_matrix.csv', delimiter=',')

    print(facility[:2])
    
    J = distance_matrix.shape[0]
    I = distance_matrix.shape[1]

    mask_1 = distance_matrix<=r 

    distance_matrix[mask_1] = 1
    distance_matrix[~mask_1] = 0

    # initiate gurobi model
    m = Model()

    # add variables 
    x = {}
    y = {}

    for i in range(I):
      y[i] = m.addVar(vtype=GRB.BINARY, name="y%d" % i)
    for j in range(J):
      x[j] = m.addVar(vtype=GRB.BINARY, name="x%d" % j)

    m.update()

    # Add constraints
    m.addConstr(quicksum(x[j] for j in range(J)) == p)

    for i in range(I):
        m.addConstr(quicksum(x[j] for j in np.where(distance_matrix[i]==1)[0]) >= y[i])

    m.setObjective(quicksum(y[i]for i in range(I)),GRB.MAXIMIZE)
    m.setParam('OutputFlag', 0)

    m.update()
    
    m.optimize()

    end = time.time()

    print('---------------------------')
    print('  Running time : %s seconds' % float(end-start))
    print('  Optimal coverage points: %g' % m.objVal)

    solution = []
    if m.status == GRB.Status.OPTIMAL:
        for v in m.getVars():
            # print v.varName,v.x
            if v.x==1 and v.varName[0]=="x":
               solution.append(int(v.varName[1:]))
    opt_sites = facility[solution,:]
    return opt_sites,m.objVal

opt_sites, obj_val = solve_exact(0.1,10)

defaultdict(<class 'set'>, {1: {27}, 2: {27, 28}, 3: {28}, 4: {46}, 5: {20, 46}, 6: {0, 20}, 7: {0, 24, 20, 15}, 8: {37, 15}, 9: {37}, 11: {17, 27}, 12: {27, 28}, 13: {28}, 14: {48, 18, 46}, 15: {48, 18, 46}, 16: {0, 20}, 17: {0, 15}, 20: {17, 30}, 21: {17}, 22: {42}, 23: {42, 5}, 24: {48, 36, 5}, 25: {48, 49, 18}, 26: {49}, 27: {4}, 29: {38}, 30: {30}, 32: {42}, 33: {42, 36, 5}, 34: {33, 36, 5, 39}, 35: {33, 49, 39}, 36: {49, 4}, 37: {4}, 39: {38, 6, 47}, 43: {9}, 44: {9, 39, 33}, 45: {33, 3, 39}, 49: {6, 47}, 51: {2}, 52: {25, 2, 26}, 53: {32, 9, 26, 25}, 54: {32, 3, 9, 12, 19, 26}, 55: {19, 3}, 56: {3}, 57: {11, 22}, 58: {11, 44}, 59: {44}, 60: {13}, 61: {2, 45}, 62: {25, 2, 10, 45}, 63: {10, 12, 25, 26, 31}, 64: {32, 12, 31}, 65: {19, 3}, 66: {7, 40, 22, 23, 29}, 67: {11, 22, 7}, 68: {44, 22}, 69: {44}, 70: {16, 13}, 71: {16, 45}, 72: {16, 1, 10, 45}, 73: {10, 43, 31}, 74: {43, 31}, 75: {40, 29, 23}, 76: {40, 23, 29, 7}, 77: {7}, 81: {16}, 82: {1, 35}, 83: {1, 35, 14, 43}, 84: {8, 

In [115]:
import numpy as np
import pickle
import gurobipy as gp
from gurobipy import GRB
import time
import sys
import re
    

def solve_exact(radius,p_tosite):
    """Solves the MCLP problem exactly using the Gurboi solver
    
    Credit: https://github.com/cyang-kth/maximum-coverage-location/blob/master/mclp.py

    Inputs: 
        r - radius for coverage 
        p - number of facilities to be sited

    """
    orig_stdout = sys.stdout
    f = open('out2.txt', 'w')
    sys.stdout = f

    start = time.time()

    dtype = [('facility', int), ('population', float), ('cnt', int)]
    facility = np.genfromtxt('facility.csv', delimiter=',', skip_header=1)

    dtype2 = [('x', float), ('y', float), ('pop', int), ('id', int)]
    demand = np.genfromtxt('demand.csv', delimiter=',', skip_header=1, dtype=dtype2)
    total_pop = np.genfromtxt('total_pop.csv', delimiter=',', skip_header=1, dtype=dtype)
    with open('covered.pickle', 'rb') as handle:
        coverage1 = pickle.load(handle)
    distance_matrix = np.genfromtxt('distance_matrix.csv', delimiter=',')
    #print(coverage1)

    dict_set = defaultdict(set)
    for k,v in coverage1.items():
        for i in v: 
            dict_set[k].add(i)
    
    #print('dict_set: ', dict_set)

    #print(facility[:2])
    dd = {}
    for i in demand:
        #print(i)
        dd[i[3]] = i[2]

    #print('dict:', dd)

    regions, population = gp.multidict(dd)
    #print('regions', [r for r in regions])
    
    J = distance_matrix.shape[0]
    I = distance_matrix.shape[1]

    m = gp.Model("cover_spam")

    build = m.addVars(J, vtype=GRB.BINARY, name="Build")
    is_covered = m.addVars(I, vtype=GRB.BINARY, name="Is_covered")

    #sites, coverage = gp.multidict({coverage1.keys(), coverage1.values()}) #dict_set.keys(), dict_set.values() #gp.multidict(dict_set)
    coverage = dict_set
    sites = np.arange(0,100)
    #print('coverage', coverage)

    m.addConstrs((gp.quicksum(build[t] for t in sites if r in coverage[t]) >= is_covered[r]
                        for r in regions), name="Build2cover")

    m.addConstr(gp.quicksum(build[t] for t in sites) <= p_tosite, name="p_tosite")
    #m.addConstr(sum(build) <= p_tosite)

    m.setObjective(is_covered.prod(population), GRB.MAXIMIZE)

    m.optimize() 

    total_population = 0

    for region in range(len(regions)):
        total_population += population[region]

    coverage_sum = round(100*m.objVal/total_population, 2)

    solution = []
    patt = r'\[(\d+)\]'
    if m.status == GRB.Status.OPTIMAL:
        for v in m.getVars():
            if v.x ==1 and 'Build' in v.varName:
                s = re.search(patt, v.varName)
                #print('%s %g' % (v.varName, v.x))
                #print(s.group(0))
                solution.append(s.group(1))
        # for v in m.getVars():
        #     if v.X==1:# and v.varName=='Is_covered':
        #         print(v.varName)
        #         print(type(v))
            #if v.x==1 and v.varName[0]=="x":
            #   solution.append(int(v.varName[1:]))
    #opt_sites = sites[solution]
    #solution = [item for sublist in solution for item in sublist]
    print(solution)

    print(f"\n The population coverage associated to the cell towers build plan is: {coverage_sum} %")

    sys.stdout = orig_stdout
    f.close()

    # print('---------------------------')
    # print('  Running time : %s seconds' % float(end-start))
    # print('  Optimal coverage points: %g' % m.objVal)

    # solution = []
    # if m.status == GRB.Status.OPTIMAL:
    #     for v in m.getVars():
    #         # print v.varName,v.x
    #         if v.x==1 and v.varName[0]=="x":
    #            solution.append(int(v.varName[1:]))
    # opt_sites = facility[solution,:]
    # return opt_sites,m.objVal

    solution = [int(i) for i in solution]

    np.savetxt('sited_opt_id.csv', solution, delimiter=',', header='facility', comments='', fmt='%i')

    return regions, population, sites, coverage

regions, population, sites, coverage = solve_exact(0.1,10)

In [98]:
help(Var)